<a href="https://colab.research.google.com/github/tomik062/AI_Project/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fetch from github

In [1]:
import requests
import os

repo_owner = 'tomik062'
repo_name = 'AI_Project'
directory_path = 'data'
url = f'https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{directory_path}'

response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes
files_in_directory = response.json()

# Create a directory to save the downloaded files
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

for item in files_in_directory:
    if item['type'] == 'file':
        download_url = item['download_url']
        file_content = requests.get(download_url).content
        file_name = os.path.join(directory_path, item['name'])

        with open(file_name, 'wb') as f:
            f.write(file_content)

        print(f"Downloaded {item['name']} to {file_name}")

Downloaded GNI_PPP_const_2021_dollars.csv to data/GNI_PPP_const_2021_dollars.csv
Downloaded Religious Composition 2010-2020 (percentages).xlsx to data/Religious Composition 2010-2020 (percentages).xlsx
Downloaded abortion-rates-by-country-2025.csv to data/abortion-rates-by-country-2025.csv
Downloaded age-at-first-marriage-by-country-2025.csv to data/age-at-first-marriage-by-country-2025.csv
Downloaded average-workweek-by-country-2025.csv to data/average-workweek-by-country-2025.csv
Downloaded female-labor-force-participation-rates-slopes.csv to data/female-labor-force-participation-rates-slopes.csv
Downloaded gender_inequality_index to data/gender_inequality_index
Downloaded happiness-cantril-ladder.csv to data/happiness-cantril-ladder.csv
Downloaded healthcare-expenditure-per-capita-ppp.csv to data/healthcare-expenditure-per-capita-ppp.csv
Downloaded high_education_female.xlsx to data/high_education_female.xlsx
Downloaded high_education_male.xlsx to data/high_education_male.xlsx
Downl

In [35]:
import pandas as pd
import io
def extract_WorldBank():
    df_list=[]
    df_list.append(extract_WorldBank_file('data/healthcare-expenditure-per-capita-ppp.csv'))
    df_list.append(extract_WorldBank_file('data/physicians-per-capita.csv'))
    df_list.append(extract_WorldBank_file('data/GNI_PPP_const_2021_dollars.csv'))
    return df_list

def extract_WorldBank_file(file):
    header_row_index = None
    with open(file, 'r', encoding='utf-8') as f:
      for i, line in enumerate(f):
          if 'Country Name' in line and 'Country Code' in line:
              header_row_index = i
              break
    if(header_row_index==None):
      print("Error: Header row not found in the file.")
      return None
    df = pd.read_csv(file, delimiter=',', engine='python', skiprows=header_row_index)
    df = df.drop(columns=['Indicator Name', 'Indicator Code', '2024', 'Unnamed: 69'],errors='ignore')
    return df

In [3]:
import pandas as pd

def extract_OurWorldInData():
    df_list=[]
    df_list.append(extract_OurWorldInData_file('data/urban-population-share.csv'))
    df_list.append(extract_OurWorldInData_file('data/period-average-age-of-mothers.csv'))
    df_list.append(extract_OurWorldInData_file('data/happiness-cantril-ladder.csv'))
    df_list.append(extract_OurWorldInData_file('data/female-labor-force-participation-rates-slopes.csv'))
    return df_list
def extract_OurWorldInData_file(file_name):
    df = pd.read_csv(file_name, delimiter=',', engine='python')
    if 'Entity' not in df.columns:
        print("Error: 'Entity' column not found. Cannot filter regions.")
        return df

    country_name_mapping = {
        'Egypt': 'Egypt, Arab Rep.',
        'Bahamas': 'Bahamas, The',
        'Brunei': 'Brunei Darussalam',
        'Democratic Republic of Congo': 'Congo, Dem. Rep.',
        'Congo': 'Congo, Rep.',
        'Cape Verde': 'Cabo Verde',
        'Gambia': 'Gambia, The',
        'Iran': 'Iran, Islamic Rep.',
        'Kyrgyzstan': 'Kyrgyz Republic',
        'Saint Kitts and Nevis': 'St. Kitts and Nevis',
        'South Korea': 'Korea, Rep.',
        'Laos': 'Lao PDR',
        'Saint Lucia': 'St. Lucia',
        'North Macedonia': 'North Macedonia',
        'Micronesia (country)': 'Micronesia, Fed. Sts.',
        'Russia': 'Russian Federation',
        'Slovakia': 'Slovak Republic',
        'Syria': 'Syrian Arab Republic',
        'East Timor': 'Timor-Leste',
        'Turkey': 'Turkiye',
        'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
        'Venezuela': 'Venezuela, RB',
        'Vietnam': 'Viet Nam',
        'Yemen': 'Yemen, Rep.',
        'Hong Kong': 'Hong Kong SAR, China',
        'Macao': 'Macao SAR, China',
        'St. Martin (French part)': 'St. Martin (French part)',
        'Puerto Rico': 'Puerto Rico (US)',
        "North Korea": "Korea, Dem. People's Rep.",
        'Palestine': 'West Bank and Gaza',
        'Kosovo': 'Kosovo',
        'United States Virgin Islands': 'Virgin Islands (U.S.)',
    }

    df['Entity'] = df['Entity'].apply(lambda x: country_name_mapping.get(x, x))


    # Reshape the DataFrame from long to wide format.
    # 'Entity' and 'Code' will be the id_vars, 'Year' will be the pivot column,
    # and the remaining column (the urbanization share) will be the value.
    # We need to find the name of the urbanization share column dynamically.
    value_vars = [col for col in df.columns if col not in ['Entity', 'Code', 'Year']]

    if not value_vars:
        print("Error: Could not find the urbanization share value column.")
        return df # Return without reshaping if value column is missing

    # Assuming there is only one value column after excluding id_vars and Year
    value_name = value_vars[0]


    # Pivot the DataFrame to wide format. Missing combinations of index and columns will result in NaN.
    df_wide = df.pivot(index=['Entity', 'Code'], columns='Year', values=value_name).reset_index()

    # Rename columns to match the fertility DataFrame structure
    df_wide = df_wide.rename(columns={'Entity': 'Country Name', 'Code': 'Country Code'})

    return df_wide

In [36]:
def merge(data, df_list):
    for new_df in df_list:
      new_df_year_columns = [col for col in new_df.columns if isinstance(col, int)]
      new_df_country_map = new_df.set_index('Country Name')
      for key, value in data.items():
          country_name = key[0]
          year = key[1]
          data_list = value[1]
          if country_name in new_df_country_map.index:
              new_df_row = new_df_country_map.loc[country_name]
              if year in new_df_row.index and pd.notna(new_df_row[year]):
                      new_value = new_df_row[year]
                      data_list.append(new_value)
              #spaghetti code incoming, hope doest break anything
              elif str(year) in new_df_row.index and pd.notna(new_df_row[str(year)]):
                      new_value = new_df_row[str(year)]
                      data_list.append(new_value)
              else:
                      data_list.append(float('nan'))

          else:
              print(f"Warning: Country '{country_name}' not found in the new DataFrame.")
              data_list.append(float('nan'))


init data for single year

In [40]:
data={}
df_fertility=extract_WorldBank_file('data/total-fertility-rate.csv')
regions_to_exclude_by_code = [
    'CEB', 'EAR', 'IBD', 'LTE', 'MAF', 'PSS', 'AFE', 'AFW', 'ARB', 'CSS', 'EAP', 'EAS', 'ECA', 'ECS', 'EMU', 'EUU', 'FCS', 'HIC', 'HPC',
    'IDA', 'IDB', 'IDX', 'LAC', 'LCN', 'LDC', 'LIC', 'LMC', 'LMY', 'MEA', 'MIC', 'MNA',
    'NAC', 'OED', 'OSS', 'PRE', 'PST', 'SAS', 'SSA', 'SSF', 'SST', 'UMC', 'WLD','IBT','TEA','TEC','TLA','TSS','TMN','TSA'
]
df_fertility = df_fertility[~df_fertility['Country Code'].isin(regions_to_exclude_by_code)]
df_fertility_2023 = df_fertility[['Country Name', '2023']].copy()
df_fertility_2023['2023'] = pd.to_numeric(df_fertility_2023['2023'], errors='coerce')
df_fertility_2023.dropna(subset=['2023'], inplace=True)
for index, row in df_fertility_2023.iterrows():
    country_name = row['Country Name']
    fertility_rate = row['2023']
    data[(country_name,2023)] = (fertility_rate, [])
merge(data,extract_OurWorldInData()+extract_WorldBank())
print(data)

{('Aruba', 2023): (1.602, [np.float64(49.899), np.float64(28.824), nan, nan, nan, nan, np.float64(41080.0)]), ('Afghanistan', 2023): (4.84, [np.float64(29.149118), np.float64(30.366), np.float64(1.721), nan, nan, nan, np.float64(2210.0)]), ('Angola', 2023): (5.124, [np.float64(66.94085), np.float64(28.589), nan, nan, nan, nan, np.float64(7280.0)]), ('Albania', 2023): (1.348, [np.float64(61.257164), np.float64(29.132), np.float64(5.304), nan, nan, nan, np.float64(20990.0)]), ('Andorra', 2023): (1.082, [np.float64(95.00263), np.float64(32.518), nan, nan, np.float64(5640.93945946982), nan, np.float64(74250.0)]), ('United Arab Emirates', 2023): (1.2, [np.float64(87.209015), np.float64(31.897), np.float64(6.733), np.float64(54.477), nan, nan, np.float64(76680.0)]), ('Argentina', 2023): (1.5, [np.float64(94.48409), np.float64(29.073), np.float64(6.188), np.float64(53.163), nan, nan, np.float64(29460.0)]), ('Armenia', 2023): (1.9, [np.float64(70.10863), np.float64(27.781), np.float64(5.455), 